In [5]:
from pathlib import Path

ROOT = Path(globals()['__vsc_ipynb_file__']).parent.parent
print(ROOT)

import sys
sys.path.append(str(ROOT))

/Users/gabrieleagliardi/Documents/GitHub/Client-Vanguard-Optimization


In [6]:
import glob
import pandas as pd
from qiskit import qpy

In [7]:
data = []
for file_name in glob.glob(f'{ROOT}/data/transpilation/**/**.qpy'):
    parsed = Path(file_name).stem.split('_')
    if len(parsed) == 7:
        parsed.append(parsed[6])
        parsed[6] = None
    
    with open(file_name, 'rb') as f:
        isa_ansatz = qpy.load(f)[0]

    data.append({
        'entanglement': Path(file_name).parent.name,
        'qubits': int(parsed[0]),
        'transpiler': parsed[1],
        'device': parsed[3],
        'ansatz': parsed[4],
        'reps': int(parsed[5]),
        'warm': bool(parsed[6]),
        'trial': int(parsed[7]),
        'depth': isa_ansatz.depth(),
        '2q-depth': isa_ansatz.depth(lambda x:x.operation.num_qubits==2),
        'count_ops': isa_ansatz.count_ops(),
        'params': isa_ansatz.num_parameters,
    })

df = pd.DataFrame(data)
df['gates'] = df.count_ops.apply(lambda x: sum(v for k,v in x.items() if k not in ['measure']))
df['2q-gates'] = df.count_ops.apply(lambda x: sum(v for k,v in x.items() if k in ['cz', 'ecr']))

df

,entanglement,qubits,transpiler,device,ansatz,reps,warm,trial,depth,2q-depth,count_ops,params,gates,2q-gates
0,di-color,109,ai,kyiv,TwoLocal,3,False,7,134,35,"{'rz': 1772, 'sx': 998, 'ecr': 369, 'x': 207, ...",436,3347,369
1,di-color,109,ai,kyiv,TwoLocal,1,False,5,88,26,"{'rz': 790, 'sx': 472, 'ecr': 123, 'measure': ...",218,1459,123
2,di-color,109,pm,kyiv,bfcd,2,True,3,141,18,"{'rz': 4191, 'sx': 2414, 'ecr': 738, 'x': 116,...",819,7460,738
3,di-color,109,pm,kyiv,TwoLocal,2,True,2,42,6,"{'rz': 1321, 'sx': 752, 'ecr': 246, 'measure':...",327,2382,246
4,di-color,109,pm,kyiv,bfcd,2,True,8,141,18,"{'rz': 4191, 'sx': 2414, 'ecr': 738, 'x': 116,...",819,7460,738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,color,155,pm,marrakesh,bfcd,1,True,6,42,9,"{'sx': 1559, 'rz': 1378, 'cz': 525, 'measure':...",505,3463,525
1046,color,109,pm,marrakesh,TwoLocal,2,True,1,19,6,"{'sx': 654, 'rz': 654, 'cz': 246, 'measure': 1...",327,1555,246
1047,color,155,ai,marrakesh,TwoLocal,2,False,0,48,35,"{'sx': 930, 'rz': 930, 'cz': 350, 'measure': 1...",465,2211,350
1048,color,109,pm,marrakesh,bfcdR,3,True,7,118,27,"{'sx': 3170, 'rz': 2759, 'cz': 1107, 'measure'...",696,7037,1107


In [8]:
df = df.sort_values(['entanglement', 'qubits', 'ansatz', 'reps', 'depth', '2q-depth', 'gates', '2q-gates'])
df['discard'] = ((df.entanglement == df.entanglement.shift()) &
            (df.qubits == df.qubits.shift()) & 
            (df.ansatz == df.ansatz.shift()) & 
            (df.reps == df.reps.shift()))

In [9]:
df_best = df[~df['discard']]
df_best[['qubits', 'entanglement', 'ansatz', 'reps', 'depth', '2q-depth', 'gates', '2q-gates', 'params']]

,qubits,entanglement,ansatz,reps,depth,2q-depth,gates,2q-gates,params
185,109,bilinear,TwoLocal,1,11,2,981,108,218
332,109,bilinear,TwoLocal,2,17,4,1525,216,327
181,109,bilinear,TwoLocal,3,23,6,2069,324,436
333,109,bilinear,bfcd,1,33,6,2547,324,434
187,109,bilinear,bfcd,2,60,12,4602,648,759
329,109,bilinear,bfcd,3,87,18,6657,972,1084
327,109,bilinear,bfcdR,1,30,6,2165,324,217
195,109,bilinear,bfcdR,2,57,12,4220,648,434
322,109,bilinear,bfcdR,3,84,18,6275,972,651
459,155,bilinear,TwoLocal,1,37,16,1829,316,310


In [10]:
df_best

,entanglement,qubits,transpiler,device,ansatz,reps,warm,trial,depth,2q-depth,count_ops,params,gates,2q-gates,discard
185,bilinear,109,pm,marrakesh,TwoLocal,1,False,2,11,2,"{'sx': 436, 'rz': 436, 'measure': 109, 'cz': 1...",218,981,108,False
332,bilinear,109,pm,marrakesh,TwoLocal,2,False,3,17,4,"{'sx': 654, 'rz': 654, 'cz': 216, 'measure': 1...",327,1525,216,False
181,bilinear,109,pm,marrakesh,TwoLocal,3,False,0,23,6,"{'sx': 872, 'rz': 872, 'cz': 324, 'measure': 1...",436,2069,324,False
333,bilinear,109,pm,marrakesh,bfcd,1,False,1,33,6,"{'sx': 1137, 'rz': 1085, 'cz': 324, 'measure':...",434,2547,324,False
187,bilinear,109,pm,marrakesh,bfcd,2,False,0,60,12,"{'sx': 2056, 'rz': 1897, 'cz': 648, 'measure':...",759,4602,648,False
329,bilinear,109,pm,marrakesh,bfcd,3,False,3,87,18,"{'sx': 2975, 'rz': 2709, 'cz': 972, 'measure':...",1084,6657,972,False
327,bilinear,109,pm,marrakesh,bfcdR,1,False,7,30,6,"{'sx': 973, 'rz': 867, 'cz': 324, 'measure': 1...",217,2165,324,False
195,bilinear,109,pm,marrakesh,bfcdR,2,False,6,57,12,"{'sx': 1892, 'rz': 1679, 'cz': 648, 'measure':...",434,4220,648,False
322,bilinear,109,pm,marrakesh,bfcdR,3,False,5,84,18,"{'sx': 2811, 'rz': 2491, 'cz': 972, 'measure':...",651,6275,972,False
459,bilinear,155,ai,marrakesh,TwoLocal,1,False,4,37,16,"{'sx': 892, 'rz': 620, 'cz': 316, 'measure': 1...",310,1829,316,False
